In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from torchvision import transforms

In [ ]:
model_name = 'resnet50'
model_path = os.path.join('..', 'fine_tuned_models', model_name)
model = torch.load(model_path)

In [ ]:
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

# Feature Extraction

In [ ]:
def extract_features(model, device, root_dir):

    # Define transformations to preprocess the input image
    transform = transforms.Compose([
        transforms.Resize((299, 299)),  # Resize to the input size of the model
        transforms.CenterCrop(299),
        transforms.ToTensor(),           # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ])


    model.eval()
    # Initialize an empty list to store rows
    rows = []

    # Loop through each subdirectory in the root directory
    for subdir in ['train', 'valid']:
        subdir_path = os.path.join(root_dir, subdir)

        # Check if it's a directory
        if os.path.isdir(subdir_path):
            # Loop through each image file in the subdirectory
            for file_name in tqdm(os.listdir(subdir_path)):
                file_path = os.path.join(subdir_path, file_name)

                # Load and preprocess the image
                image = Image.open(file_path).convert('RGB')
                input_tensor = transform(image).unsqueeze(0)  # Add a batch dimension
                # Move the input tensor to the appropriate device (GPU if available)
                input_tensor = input_tensor.to(device)

                # Forward pass
                with torch.no_grad():
                    output = model(input_tensor)

                # Extract features
                feats = np.squeeze(output).cpu().numpy().flatten()
                # Append image name and features to the list
                row_data = [file_name] + list(feats)
                rows.append(row_data)

    # Create a DataFrame
    result_df = pd.DataFrame(rows)

    return result_df

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
device

In [ ]:
# Define data directory
district = 'gujranwala'
data_dir = os.path.join('..', 'data', 'images', district)
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")

In [ ]:
# Define data directory
district = 'gujranwala'
imgs_path = os.path.join('..', 'data', 'images', district)
img_feats = extract_features(feature_extractor, device, imgs_path)

In [ ]:
img_feats.rename({0:'cluster_name'}, axis=1, inplace=True)
img_feats['cluster_name'] = img_feats['cluster_name'].str.rsplit('_', n=1).str[0]
img_feats

In [ ]:
img_feats['cluster_name'][0]

In [ ]:
df_path = os.path.join('..', 'data', 'processed', 'finalized_df.csv')
df = pd.read_csv(df_path)
df.sample(10)

In [ ]:
joined_df = df.set_index('cluster_name').join(img_feats.set_index('cluster_name'), how='inner').reset_index(drop=True)
joined_df.sample(10)

In [ ]:
data_save_path = os.path.join(model_path, 'processed_data')
os.makedirs(data_save_path, exist_ok=True)
file_path = os.path.join(data_save_path, 'feature_indexed_' + model_name + '.csv')
joined_df.to_csv(file_path, index=False)

# Data Pre-Processing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
model_path = os.path.join('..', 'fine_tuned_models', model_name)

In [ ]:
model_name = 'resnet50'
root_path = os.path.join('..', 'fine_tuned_models', model_name, 'processed_data')
file_path = os.path.join(root_path, 'feature_indexed_' + model_name + '.csv')
df = pd.read_csv(file_path)
df.sample(10)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.drop([
    'district', 'dist_lat', 'dist_lon', 'cluster_lat',
    'cluster_lon', 'nightlights', 'labels'
], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['income_per_cluster'].is_monotonic_increasing

In [ ]:
plt.boxplot(df['income_per_cluster'])

In [ ]:
def replace_outliers_with_closest(data_):
    # Calculate the inter quartile range (IQR)
    q1 = np.percentile(data_, 25)
    q3 = np.percentile(data_, 75)
    iqr = q3 - q1

    # Define the lower and upper bounds for outliers
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Replace outliers with the closest values within the IQR
    replaced_data = np.where(data_ < lower_bound, q1, data_)
    replaced_data = np.where(replaced_data > upper_bound, q3, replaced_data)

    return replaced_data

In [ ]:
corrected_income = replace_outliers_with_closest(df['income_per_cluster'])

In [ ]:
plt.boxplot(corrected_income)

In [ ]:
df['income_per_cluster'] = corrected_income

In [ ]:
df['income_per_cluster'].is_monotonic_increasing

In [ ]:
df = df.sort_values(by='income_per_cluster').reset_index(drop=True)

In [ ]:
df['income_per_cluster'].is_monotonic_increasing

In [ ]:
df.to_csv(os.path.join(root_path, 'preprocessed_df_' + model_name + '.csv'), index=False)

# Model Training - Income Prediction

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
fig_path = os.path.join('..', 'fine_tuned_models', model_name, 'figures')
os.makedirs(fig_path, exist_ok=True)

In [ ]:
model_name = 'resnet50'
root_path = os.path.join('..', 'fine_tuned_models', model_name, 'processed_data')
file_path = os.path.join(root_path, 'preprocessed_df_' + model_name + '.csv')
df = pd.read_csv(file_path)
df.sample(10)

In [ ]:
df['income_per_cluster'].is_monotonic_increasing

In [ ]:
plt.boxplot(df['income_per_cluster'])

In [ ]:
scaler = StandardScaler()

X, y = df.drop('income_per_cluster', axis=1), df['income_per_cluster']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)

# X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
# X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

y_train = scaler.fit_transform(y_train.values.reshape(-1,1))
y_test = scaler.transform(y_test.values.reshape(-1,1))

ridge_model = Ridge(alpha=1)
ridge_model.fit(X_train, y_train)

y_test_pred = ridge_model.predict(X_test)
print(f'R2 score: {r2_score(y_test, y_test_pred)}')

plt.scatter(y_test, y_test_pred, alpha=.7)

# Plot the perfect prediction line
min_val = min(np.min(y_test), np.min(y_test_pred))
max_val = max(np.max(y_test), np.max(y_test_pred))
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)  # Perfect prediction line

# Labeling
plt.title('Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')

plt.savefig(os.path.join(fig_path, 'ridge_model'))
plt.show()

# Applying K-fold

In [ ]:
def visualize_results(tr_targets, tr_preds, te_targets, te_preds, fig_path):

    # Calculate R2 scores
    train_r2 = r2_score(tr_targets, tr_preds)
    test_r2 = r2_score(te_targets, te_preds)

    # Plotting
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Visualize the model on the training set
    axs[0].scatter(tr_targets, tr_preds, alpha=0.5)
    axs[0].plot([min(tr_targets), max(tr_targets)], [min(tr_targets), max(tr_targets)], linestyle='--', color='red',
                label='R2 line')
    axs[0].set_xlabel('Actual Values')
    axs[0].set_ylabel('Predicted Values')
    axs[0].set_title(f'Training Set\nR2: {train_r2:.2f}')
    axs[0].legend()

    # Visualize the model on the test set with R^2 score
    axs[1].scatter(te_targets, te_preds, alpha=.5)
    axs[1].plot([min(te_targets), max(te_targets)], [min(te_targets), max(te_targets)], linestyle='--', color='red',
                label='R2 Line')
    axs[1].set_xlabel('Actual Values')
    axs[1].set_ylabel('Predicted Values')
    axs[1].set_title(f'Test Set\nR2 Score: {test_r2:.2f}')
    axs[1].legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()
    # Show the plots
    plt.savefig(fig_path)
    plt.show()

In [ ]:
def train_and_test_kfold(data, fig_path_, alpha=1, k=5):
    scaler = StandardScaler()
    #     data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    X, y = data.drop('income_per_cluster', axis=1), data['income_per_cluster']

    ridge_model = Ridge(alpha=alpha)

    # Perform k-fold cross-validation
    kf = KFold(n_splits=k, shuffle=True)

    all_actual_test_values = []
    all_predicted_test_values = []
    all_actual_train_values = []
    all_predicted_train_values = []

    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

        y_train = scaler.fit_transform(y_train.values.reshape(-1,1))
        y_test = scaler.transform(y_test.values.reshape(-1,1))

        ridge_model.fit(X_train, y_train)

        # Training set MSE and R-squared
        y_train_pred = ridge_model.predict(X_train)
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        print(f'Fold {fold} - Training Mean Squared Error: {train_mse:.4f}, Training R-squared Score: {train_r2:.4f}')

        # Test set MSE and R-squared
        y_test_pred = ridge_model.predict(X_test)
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        print(f'Fold {fold} - Test Mean Squared Error: {test_mse:.4f}, Test R-squared Score: {test_r2:.4f}\n')

        # Save actual and predicted values
        all_actual_test_values.extend(y_test)
        all_predicted_test_values.extend(y_test_pred)
        all_actual_train_values.extend(y_train)
        all_predicted_train_values.extend(y_train_pred)

    # Overall R-squared
    overall_r2 = r2_score(all_actual_train_values, all_predicted_train_values)
    print(f'Overall/Average train R-squared Score: {overall_r2:.4f}\n')
    overall_r2 = r2_score(all_actual_test_values, all_predicted_test_values)
    print(f'Overall/Average train R-squared Score: {overall_r2:.4f}\n')

    fig_save_path = os.path.join(fig_path_)

    visualize_results(
        all_actual_train_values,
        all_predicted_train_values,
        all_actual_test_values,
        all_predicted_test_values,
        fig_save_path
    )

In [ ]:
train_and_test_kfold(df, os.path.join(fig_path, 'kfold_ridge'), alpha=1)

# Using Vanilla Neural Network

In [ ]:
import torch
import joblib
import pandas as pd
from torch import nn
import torch.optim as optim
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold

In [ ]:
root_path = os.path.join('..', 'fine_tuned_models', model_name, 'processed_data')
file_path = os.path.join(root_path, 'preprocessed_df_' + model_name + '.csv')
df = pd.read_csv(file_path)

In [ ]:
model_name = 'resnet50'
model_path = os.path.join('..', 'fine_tuned_models', model_name)

In [ ]:
# Define the neural network model
class SimpleRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(SimpleRegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [ ]:
def preprocess_and_create_dataloader_kfold(X, y, b_size=16):
    # Apply standard scaling
    scaler = StandardScaler()

    # Apply scaling to features
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    y_scaled = scaler.fit_transform(y.values.reshape(-1,1))

    # Convert NumPy arrays to PyTorch tensors
    X_tensor = torch.FloatTensor(X.values)
    y_tensor = torch.FloatTensor(y_scaled)  # Reshape to ensure it's a column vector

    # Create DataLoader
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=b_size, shuffle=True)

    return dataloader, X_tensor.shape[1], scaler

In [ ]:
# Training function
def train(loader_train, loader_val, input_size, device, num_epochs=20, set_seed=False, lr=0.001, visualize=True, print_rslts=True):
    if set_seed:
        # Set random seeds for reproducibility
        seed = 42
        torch.manual_seed(seed)
        np.random.seed(seed)

    # Instantiate the model, define loss function, and optimizer
    model = SimpleRegressionModel(input_size=input_size).to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    #     criterion = nn.MSELoss()
    #     optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_weights = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for batch_x, batch_y in loader_train:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(loader_train)
        train_losses.append(avg_train_loss)

        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for val_x, val_y in loader_val:
                val_x, val_y = val_x.to(device), val_y.to(device)
                val_outputs = model(val_x)
                val_loss += criterion(val_outputs, val_y).item()

        avg_val_loss = val_loss / len(loader_val)
        val_losses.append(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_weights = model.state_dict().copy()

        if print_rslts:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

    if visualize:
        # Visualize the Error lines
        plt.plot(range(1, num_epochs + 1), train_losses, color='red', marker='o', label='Train Error')
        plt.plot(range(1, num_epochs + 1), val_losses, color='blue', marker='o', label='Val Error')
        plt.xlabel('Epochs')
        plt.ylabel('Value')
        plt.title('Error During Training and Validation')
        plt.legend()
        plt.show()

    # Load the best model weights
    model.load_state_dict(best_model_weights)

    return model, best_val_loss

In [ ]:
# Evaluation function
def evaluate(loader_test, model, device):
    model.eval()
    total_loss = 0.0
    criterion = nn.L1Loss()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_x, batch_y in loader_test:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
            all_predictions.extend(outputs.detach().cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())

    avg_loss = total_loss / len(loader_test)
    return all_targets, all_predictions, avg_loss

In [ ]:
# K-fold cross-validation function with print statement for best model
def k_fold_cross_validation(df, figure_path, model_path, print_rslts=False, visualize=False, k=5, num_epochs=20, lr=0.001, batch_size=16):
    # Get features and target
    X, y = df.drop('income_per_cluster', axis=1), df['income_per_cluster']

    # Initialize KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    all_train_targets = []
    all_train_predictions = []
    all_test_targets = []
    all_test_predictions = []
    all_test_r2 = []
    all_train_r2 = []
    best_val_loss = float('inf')
    best_model = None
    best_fold = -1
    scaler = None

    # Define device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for fold_idx, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Preprocess and create data loaders
        train_loader, size_input, scaler = preprocess_and_create_dataloader_kfold(X_train, y_train, b_size=batch_size)
        val_loader, _, _ = preprocess_and_create_dataloader_kfold(X_val, y_val, b_size=batch_size)

        # Train the model
        trained_model, fold_best_val_loss = train(loader_train=train_loader, loader_val=val_loader,
                                                  input_size=size_input, device=device, num_epochs=num_epochs, lr=lr,
                                                  print_rslts=print_rslts, visualize=visualize)

        # Check if this fold's model is the best one
        if fold_best_val_loss < best_val_loss:
            best_val_loss = fold_best_val_loss
            best_model = trained_model
            best_fold = fold_idx + 1  # Folds are 0-indexed, so add 1 for display
            print(f"Best model updated at fold {best_fold} with validation loss: {best_val_loss}")

        # Evaluate the model on train data
        train_targets, train_predictions, _ = evaluate(loader_test=train_loader, model=trained_model, device=device)
        train_r2 = r2_score(train_targets, train_predictions)
        all_train_targets.extend(train_targets)
        all_train_predictions.extend(train_predictions)
        all_train_r2.append(train_r2)

        # Evaluate the model on validation data
        val_targets, val_predictions, _ = evaluate(loader_test=val_loader, model=trained_model, device=device)
        val_r2 = r2_score(val_targets, val_predictions)
        all_test_targets.extend(val_targets)
        all_test_predictions.extend(val_predictions)
        all_test_r2.append(val_r2)

        print(f"Fold {fold_idx + 1} - Training Mean Squared Error: {mean_squared_error(train_targets, train_predictions)}, Training R-squared Score: {train_r2:.2f}")
        print(f"Fold {fold_idx + 1} - Validation Mean Squared Error: {mean_squared_error(val_targets, val_predictions)}, Validation R-squared Score: {val_r2:.2f}\n")

    print(f'Average Validation R2: {np.mean(all_test_r2):.2f}')
    print(f'Average Train R2: {np.mean(all_train_r2):.2f}')

    visualize_results(all_train_targets, all_train_predictions, all_test_targets, all_test_predictions, fig_path=figure_path)

    # Save the best model
    best_model_path = os.path.join(model_path, 'best_Regression_model.pth')
    torch.save(best_model, best_model_path)
    print(f'Best model from fold {best_fold} saved to {best_model_path}')

    scaler_path = os.path.join(model_path, 'scaler.pkl')
    joblib.dump(scaler, scaler_path)
    print(f'Scaler saved to {scaler_path}')

    return all_train_targets, all_train_predictions, all_test_targets, all_test_predictions

In [ ]:
# Perform k-fold cross-validation
train_targets, train_predictions, test_targets, test_predictions = k_fold_cross_validation(df, figure_path=os.path.join(fig_path, model_name+'_NN'), model_path=model_path, num_epochs=150, batch_size=16)